### Set autoreloading
This extension will automatically update with any changes to packages in real time

In [ ]:
%load_ext autoreload
%autoreload 2

### Import packages
We'll need the `pytorch_lightning` and `nugraph` packages imported in order to train

In [ ]:
import nugraph as ng
import pytorch_lightning as pl

### Configure data module
Declare a data module. Depending on where you're working, you should edit the data path below to point to a valid data location.

In [ ]:
nudata = ng.data.H5DataModule(data_path='/raid/uboone/NuGraph2/NG2-paper.gnn.h5', batch_size=64)

### Configure network
Declare a model. You can edit the arguments below to change the network configuration.

In [ ]:
nugraph = ng.models.NuGraph2(
    in_features=4,
    planar_features=128,
    nexus_features=32,
    vertex_aggr='lstm',
    vertex_lstm_features=64,
    vertex_mlp_features=[64],
    planes=nudata.planes,
    semantic_classes=nudata.semantic_classes,
    event_classes=nudata.event_classes,
    num_iters=5,
    event_head=False,
    semantic_head=True,
    filter_head=False,
    vertex_head=False,
    checkpoint=True,
    lr=0.001)

### Configure logger and callbacks
Declare a tensorboard logger and define the output directory, so we can monitor network training. Also define a callback so we can monitor learning rate evolution.

In [ ]:
logger = pl.loggers.TensorBoardLogger(save_dir='/raid/vhewes/logs', name='semantic', version='64_16')
callbacks = [ pl.callbacks.LearningRateMonitor(logging_interval='step') ]

### Declare trainer and run training
First we set the training device. In the instance that we're in a multi-GPU environment, the code will automatically select the GPU with the most available memory; otherwise, it defaults to CPU training. We then instantiate a PyTorch Lightning trainer that we'll use for training, and then run the training stage, which iterates over all batches in the train and validation datasets to optimise model parameters, writing output metrics to tensorboard.

In [ ]:
accelerator, devices = ng.util.configure_device()
trainer = pl.Trainer(accelerator=accelerator,
                     devices=devices,
                     max_epochs=80,
                     logger=logger,
                     callbacks=callbacks)
trainer.fit(nugraph, datamodule=nudata)
trainer.test(datamodule=nudata)